# voor een nieuw tectonic moet men een file aanmaken
de eerste regel b.v. 
### nrKol, nrRij = 9, 9
aantal kolommen, aantal rijen
## voor elk vak de Coords gevolgd door string data 0 voor niet bekend
## in dezelfde volgorde als de gegeven Coords
### voorbeeld a1b1b2c2d2 00210  de letters mogen ook uit {A,B,...} zijn
# <font color='red'>see the example</font> 

<img src = '7julitub9x9.jpg'/>

# this puzzel converted to a possible inputfile:
```
nrKol, nrRij = 9, 9
a1a2b1b2b3 03200
c1c2c3d2e2 00000
d1e1f1g1h1 05000
i1i2f2g2h2 12000
a3a4 00
d3d4c4e3f3 04000
g3g4g5g6g7 03000
h3h4i3i4i5 00100
b4b5c5d5d6 00000
e4e5e6f4f5 10000
h5h6h7h8g8 00030
a5a6a7a8b7 03000
b6c6c7d7d8 00000
f6f7f8e7e8 00300
i6i7i8i9 0000
a9b8b9c8c9 21000
d9e9f9g9h9 00010
```

In [1]:
%ls 7*.

 Volume in drive C has no label.
 Volume Serial Number is E491-19A0

 Directory of C:\Users\Zbook\PKHG\tectonic\doc-master

07-07-2020  07:42               315 7julitub9x9
07-07-2020  07:33               315 7julitub9x9~
               2 File(s)            630 bytes
               0 Dir(s)  348.418.277.376 bytes free


In [2]:
filename = input("welkbestand bevat het puzzel? ")


welkbestand bevat het puzzel? 7julitub9x9


In [45]:
from z3 import *    # de microsoft solver z3 wordt gebruikt!!!
import numpy as np  #om later gekleurde oplossing te laten zien

def arithmrij(i):                   #zal voor de som getallen in een vak worden gebruikt
    return int(((i + 1) / 2 ) * i)

# om A1 ... in np coordinaten te converteren
def coord_to_np_coord(w = 'A1'):
    yc , xc = w[0], int(w[1:]) - 1
    return(xc, ord(yc)- 65)

########### kleuren gebruiken voorbereiding
from IPython.utils import coloransi
#from IPython.core import prompts
termcolors = coloransi.TermColors() # the color table
#dark = prompts.PColLinux.colors
#light = prompts.PColLightBG.colors
#dir(coloransi.TermColors)
kleurdata = {
    'Black': 40, 'Red' : 41, 'Green' :42, 'Yellow': 43,
    'Blue' : 44, 'Magenta' :45, 'Cyan':46,   'White':47}
# NN print(kleurdata.keys(), kleurdata.values())
wijzig = ['\x1b[0;' + str(el) + 'm' for el in kleurdata.values()]
printKleur = dict([])
for i, kleur in enumerate(kleurdata.keys()):
    printKleur[kleur] = wijzig[i]
#printKleur


In [47]:
fp = open(filename,'r')
alllines = fp.readlines()
alllines = [el.rstrip() for el in alllines]

exec(alllines[0])
print('nrKol = {0}  nrRij = {1}'.format(nrKol, nrRij))
#DBG print(len(alllines))
alllines.pop(0)
#DBG print(len(alllines))
#DBG nrKol, nrRij

import re
p = re.compile("[A-Z][0-9]*") #normal max range was A1..I9
def get_Coords(ts):
    """
    Short input splitted into (chess-like) coordinates
    """
    ts = ts.upper()
    iterator = p.finditer(ts)
    result = []
    for match in iterator:
        res = match.span()
        result.append(ts[res[0]:res[1]])
    return result

#print("example gegenereerde Coords", get_Coords("A1d1B2C2y2"))
#maak een dictionary per rij een dictionary
      
vak = dict([])
localVakGetallen = dict([])
#"""
#wijzig
vakkleur = 1
for i, line in enumerate(alllines):
    #PKHG???5juli 
    vakkleur = 1 + (vakkleur % (len(kleurdata) - 1))  #PKHG????
    # index error  vakkleur = 1 + (vakkleur % (len(kleurdata)))
    d0, d1 = line.split(' ')
    # print('DBG',d0, ' &&&& ', d1)
    tmp = get_Coords(d0)
    npcoords = [ coord_to_np_coord(el) for el in tmp ]
    #DBG  print("DBG tmp L34", tmp)
    localVakGetallen = [0 for i in range(len(tmp))]# = { tmp[el]: 0 for el in tmp} #om een oplossing te hebben, zelfde volgprd
    #print('line {0} line = {1} dan {2}'.format(i,tmp, d1))
    #vak['vak' + str(i)] = [tmp, d1]
    ssom = arithmrij(len(tmp))
    vak[i] = {'coords':tmp, 'waarden':d1, 'somv': ssom  , 'npcoords': npcoords,
             'kleur':wijzig[vakkleur] , 'getallen' : localVakGetallen} #python zero based nummering!!
# van example0
#print("Alle vakken opgeslaan in vak van example0 ")
#vak
#"""
fp.close()

nrKol = 9  nrRij = 9


In [26]:
def gegeven_waarden(puzzel = vak, DBG = False):
    """
    maak een lijst van de in de puzzel gegeven waarden
    """
    gegevenData = dict([])
    for i, el in enumerate(list(puzzel.values())):
        coords = el['coords']
        waarden = [ int(i) for i in el['waarden']]
        #print(coords, waarden)
        eval_dit = [(a,b) for a,b in zip(coords, waarden)]
        if len(eval_dit) == 1:
            if DBG:
                print(eval_dit[0] , 'wordt', 1)
            gegevenData[eval_dit[0][0]] = 1
        else:
            for coor, waarde in eval_dit:
                if waarde > 0:
                    if DBG:
                        print(coor, 'wordt', waarde)
                    gegevenData[coor] = waarde
                #print(coor, waarde)
        #print(eval_dit)
        #print (i, el['coords'], el['waarden'])
    return gegevenData

print(gegeven_waarden(vak)) #OLD ;-) voor exa3x3 moet dit {} opleveren 4 juli 8:07 OK
    

{'A2': 3, 'B1': 2, 'E1': 5, 'I1': 1, 'I2': 2, 'D4': 4, 'G4': 3, 'I3': 1, 'E4': 1, 'H8': 3, 'A6': 3, 'F8': 3, 'A9': 2, 'B8': 1, 'G9': 1}


In [48]:
#from z3 import *
def conditievakken(vak):
    """
    coords in een vak moeten verschillend (Distinct) zijn
    """
    start_condities = []
    for i, el in enumerate(list(vak.values())):
        coords = el['coords']
        waarden = [ int(i) for i in el['waarden']]
        len_coords = len(coords)
        #print(coords, waarden)
        
        ### PKHG_TODO de bijbehorende cell hier al op 1 zetten?!!! 4 juli 8:11 PKHG
        
        if len(coords) == 1:
            continue
        tmp = Distinct([Int(naam) for naam in coords])
        start_condities.append(tmp)
        for naam in coords:
            tmp = Int(naam)
            start_condities.append( And(tmp > 0, tmp <= len_coords))
        #print(tmp)
        #start_condities.extend([Distinct[ Int(naam) for naam in coords]])
    return start_condities    
#DBG print(conditievakken(vak))

In [49]:
#dbg len(vak)

In [51]:
all_coords = set({})
for i in range(len(vak)):
    for cors in vak[i]['coords']:
        all_coords.add(cors)
if len(all_coords) != nrKol * nrRij:
    print("ERROR in invoer, niet genoeg coordinaten gevonden")
    print("check now e.g. all_coords")
    import sys
    sys.exit()
else:
    print("Alle coordinaten gegeven ;-) ")

Alle coordinaten gegeven ;-) 


In [31]:
class Tectonic_start:
    """
    de puzzel met alle benodigede waarden heet nu vak en is intussen aangemaaket
    voor de eerste test wordt het boven gemaakte puzzel gebruikt,
    door davoor  in __init__ de waarden als default waarden te nemen
    
    het zijn dus: vak, nrRij en nrKol
    er is een debug mogelijk ingeboud te activern via een specifieke DBG = "????"
    
    """
    # PKHG ???? puzzel dict gebruiken? global vakkenlijst
    def __init__(self, vak, nrRij = nrRij, nrKol = nrKol, DBG = ' '):
        # input waarden Class variablen maken en andere voorbereidingen
        self.DBG = DBG
        self.vak = vak
        self.lenvaknamen = 1
        self.nrRij = nrRij
        self.nrKol = nrKol
        self.todolist = []
        self.oplossingen = dict()
        self.sol = None # changed in run ...   # de z3 Solver wordt gebruikt
        self.inbetween = None
        self.startconditions = []
        self.solution_models = []
        # de coordintaten (a la chess A1 ... I9 9x9 puzzel)
        self.kolnamen = [chr(65 + i) for i in range(nrKol)]    # dus een lijst van kolom namen ['A', 'B', ...]
        self.rijnamen = [chr(49 + i) for i in range(nrRij)]    # net zo voor de rijen
        self.allElements = [ a + b for a in self.kolnamen for b in self.rijnamen]  # de namen voor elkce cell zijn dus nu klaar
        
        # omdat alle vakken gebruikt moeten worden is een lijst van vakken nodig:
        #self.vakkenlijst = [el['coords'] for el in puzzel()]
        #daarmee wordt een maakelijk bruikbaar dictionary gemaakt
        
        # s a dictionary met keys de namen van de vakken values de Int van de bijbehorende naam
        self.s = {e: Int(e) for e in self.allElements}
        
        # uit het gegeven puzzel de gegeven waaren halen ==> een dict
        self.gegevenWaarden = dict()
        self.vakcondities = conditievakken(vak)  # voor de solver een lijst van tectonic condities gebruiken:
        self.strwaarden = []
        self.strnamen = []
        #self.gegevenData = dict([])
        self.gegevenData = gegeven_waarden(self.vak)
        self.startconditions = conditievakken(self.vak)
       
    def verschuifInfo_algemeen(self, r, s, nrRij  = nrRij, nrKol = nrKol):
        """
        r is (zero based) rij index,  s is (zero based) kol index het CENTRUM element
        nrRij het aantal rijen, nrKol aantal kolommen
        result  [centrum [lijst omringende cellen]]
        """
        #r += 1   #namen zijn om een verschoven met zero based real coordinates
        #s += 1
        res = [(-1 + s , r - 1 + i) for i in range(3)]
        res.extend([(0 + s, r - 1 + i) for i in range(3)])
        res.extend([(1 + s, r - 1 + i) for i in range(3)])
        result = [ chr(65 + x ) + str(y + 1) for x, y in res if x > -1 and y > -1 and x < nrKol and y < nrRij]
        midden = chr(65 + s ) + str(r + 1)
        result.pop(result.index(midden))
        return (midden, result)
    #nrRij , nrKol
    
    def make_todolist(self):
        """
        Berekening van alle vakken moeten verschillend cellen bevatten...
        """
        selftodolist = []
        for i in range(self.nrRij):
            for j in range(self.nrKol):
                #print(i,j, self.verschuifInfo_algemeen(i,j))
                selftodolist.append(self.verschuifInfo_algemeen(i,j))
        for naam, lis in selftodolist:
            for tweede in lis:
                self.todolist.append(self.s[naam] != self.s[tweede])
                #print('DBGl82', self.todolist[-1])
        return self.todolist
    
    def run_z3(self, countmax = 2):
        """
         z3 is gebruikt om te zien of alle condities kloppen
         countmax is default op 2 gezet, om te zien of een oplossig
         wel of niet eenduidig is.
        """
        self.todolist = self.make_todolist()
        #print("DBGL81 todolist = {0}".format(str(self.todolist)))
        self.sol = Solver()
        
        # DBG L95 self.gegevenWaarden {'F1': 3, 'F3': 4, 'A2': 5, 'C4': 3, 'E5': 2, 'F5': 3}
        for el in self.gegevenData.keys(): #self.gegevenWaarden.keys():
            if self.DBG == 'gegevens':
                print("DBG L91 self.gegevenWaarden", self.gegevenData)
                print("DBG L92 el", el, self.s[el] ==  self.gegevenData[el])
            self.sol.add(self.s[el] == self.gegevenData[el])
        for el in self.todolist:
            self.sol.add(el)
        #self.sol.add([self.make_todolist()])
        for el in self.startconditions:
            self.sol.add(el)
        count = 0
        while self.sol.check() == sat and count < countmax:
            count += 1
            if self.DBG == 'oplossing counter':
                print('oplossing {0} '.format(str(count)))
            m = self.sol.model()
            self.solution_models.append(m)
            r = [el + ' = ' + str(m.evaluate(self.s[el])) for el in self.allElements]
            self.oplossingen[count] = r
            if self.DBG == 'oplossing direct':
                print('de oplossing van boven naar beneden', r)
            
            vals = [ self.s[el] ==  m.evaluate(self.s[el]) for i, el in enumerate(self.allElements) ]
            #[self.A[i][j] == m.evaluate(self.A[i][j]) for j in range(size)  for i in range(size)]
            self.sol.add(Not(And(vals)))
            """
            for nietdit in vals:
                self.sol.add(nietdit)
            print ('DBG L122 deze dus niet', vals)
            """
            #DBG self.inbetween = self.sol
        #DBG return self.sol
    
   #DBG return self.s, self.gegevenWaarden, self.vakcondities, self.sol



vak

In [56]:
rt = Tectonic_start(vak,  DBG = 'oplossing direct')  
# DBG :laat de gevonden  waarden (per oplossing) zien, zonder DBG =  , dan geen uitvoer 
laatsteRES = rt.run_z3(countmax = 36)   # een andere invoer had 'veel' oplossingen
print('\n aantal gevonden oplossingen', len(rt.oplossingen))

de oplossing van boven naar beneden ['A1 = 1', 'A2 = 3', 'A3 = 1', 'A4 = 2', 'A5 = 1', 'A6 = 3', 'A7 = 4', 'A8 = 5', 'A9 = 2', 'B1 = 2', 'B2 = 4', 'B3 = 5', 'B4 = 3', 'B5 = 4', 'B6 = 5', 'B7 = 2', 'B8 = 1', 'B9 = 3', 'C1 = 5', 'C2 = 3', 'C3 = 2', 'C4 = 1', 'C5 = 2', 'C6 = 3', 'C7 = 4', 'C8 = 5', 'C9 = 4', 'D1 = 2', 'D2 = 4', 'D3 = 5', 'D4 = 4', 'D5 = 5', 'D6 = 1', 'D7 = 2', 'D8 = 1', 'D9 = 3', 'E1 = 5', 'E2 = 1', 'E3 = 3', 'E4 = 1', 'E5 = 2', 'E6 = 3', 'E7 = 5', 'E8 = 4', 'E9 = 5', 'F1 = 3', 'F2 = 4', 'F3 = 2', 'F4 = 5', 'F5 = 4', 'F6 = 1', 'F7 = 2', 'F8 = 3', 'F9 = 2', 'G1 = 1', 'G2 = 5', 'G3 = 1', 'G4 = 3', 'G5 = 2', 'G6 = 5', 'G7 = 4', 'G8 = 5', 'G9 = 1', 'H1 = 4', 'H2 = 3', 'H3 = 4', 'H4 = 5', 'H5 = 4', 'H6 = 1', 'H7 = 2', 'H8 = 3', 'H9 = 4', 'I1 = 1', 'I2 = 2', 'I3 = 1', 'I4 = 3', 'I5 = 2', 'I6 = 3', 'I7 = 4', 'I8 = 1', 'I9 = 2']

 aantal gevonden oplossingen 1


In [54]:
#DBG rt.todolist

rt.gegevenData

In [33]:
rt.gegevenWaarden

{}

In [34]:
def getallen_voor_vak(rt = rt, vak = vak, model_nr = 0, DBG = False):
    """
      zet de oplossing in het puzzel 
    """
    try:
        res = rt.solution_models[model_nr]
        len_vak = len(vak)
        for i in range(len_vak):
            riji = [ res.evaluate(rt.s[el]) for el in vak[i]['coords']]
            if DBG:
                print(i, riji)
            vak[i]['getallen'] = riji
    except:
        print("of er is geen oplossing of ...")
    return vak
#getallen_voor_vak(model_nr = 0)

In [35]:
"""
Foreground colors
       30    Black
       31    Red
       32    Green
       33    Yellow
       34    Blue
       35    Magenta
       36    Cyan
       37    White
 
    Background colors
       40    Black
       41    Red
       42    Green
       43    Yellow
       44    Blue
       45    Magenta
       46    Cyan
       47    White
"""

"""
kleurdata = {
    'Black': 40,
        'Red' : 41,
        'Green' :42,
        'Yellow': 43,
          'Blue' : 44,
          'Magenta' :45,
          'Cyan':46,
            'White':47}
print(kleurdata.keys(), kleurdata.values())
wijzig = ['\x1b[0;' + str(el) + 'm' for el in kleurdata.values()]
wijzig
"""

print('\x1b[0;34m', 'ik ben blue', '\x1b[0;45m', 'background magenta')

 ik ben blue  background magenta


In [36]:
#rt.oplossingen.values()

str(rt.solution_models[0].get_universe).split(' of ')

In [37]:
solutionDictViaCords = dict([])

In [38]:
print(rt.nrRij,rt.nrKol)
nog0 = np.zeros((nrRij,nrKol), dtype = '<U9')
#nog0

9 9


def coord_to_np_coord(w = 'A1'):
    yc , xc = w[0], int(w[1:]) - 1
    return(xc, ord(yc)- 65)
coord_to_np_coord('G5')

In [39]:
#print(toinsert[1])
if False:
    for nrdeze in range(1,  len(rt.oplossingen) + 1): #len(toinsert) + 1):     
        #print(oplossingdeze)
        #continue 6 matrices 2 x 3 met 0 gevuld
        #print(rt.oplossingen[nrdeze])
        gebruiknu = rt.oplossingen[nrdeze]
        #print(gebruiknu)
        oplossingdeze = np.zeros((nrRij, nrKol), dtype = int)
        #print(oplossingdeze)
        teller = -1
        for i in range(nrRij):
            for j in range(nrKol):
                teller += 1
                #tmp = int(gebruiknu[teller][-1])
                naam, getalnaam = gebruiknu[teller].split(' = ')
                
                #print(coord_to_np_coord(naam), getalnaam)
                x , y = coord_to_np_coord(naam)
                oplossingdeze[x,y] = getalnaam
        print(nrdeze, '\n', oplossingdeze)


# convert a numpy array into a html-table

print(coloransi.TermColors.Green, "peter", coloransi.TermColors.Red, "ikke")
print(printKleur['White'] + str(4))

In [40]:
alleklekleuren = list(kleurdata.keys())

alleklekleuren

['Black', 'Red', 'Green', 'Yellow', 'Blue', 'Magenta', 'Cyan', 'White']

In [41]:
def coord_to_np_coord(w = 'A1'):
    yc , xc = w[0], int(w[1:]) - 1
    return(xc, ord(yc)- 65)
#coord_to_np_coord('G5')
helper1 = dict([])
for co in list(rt.s.keys()):
    helper1[co] = coord_to_np_coord(co)
    #print(co , ' ==> ', coord_to_np_coord(co))

In [42]:
def generate_vak_nr_per_rij(rt = rt, nrOplossing = 1):
    opl = rt.oplossingen[nrOplossing]
    vak = rt.vak
    #print(rt.allElements)
    coordsRijen = [[rt.allElements[i] for i in range(j, nrKol*nrRij, nrKol)] for j in range(nrRij)]
    #print("coordinaten per rij", coordsRijen)
    #print("de oplossing = ", opl)
    for i in range(len(vak)):
        #print('vak{} ='.format(i),vak[i]['coords'])
        for coor in vak[i]['coords']:
            #print(coor, helper1[coor])
            pass
generate_vak_nr_per_rij()

In [43]:
def print_oplossing(model_nr = 0):
    #getallen_voor_vak(model_nr = 1)
    getallen_voor_vak(model_nr = model_nr) # wijzigd vak
    np_oplossing = np.empty((nrRij,nrKol), dtype = '<U10')
    np_oplossing
    for i in range(len(vak)):
        #print(vak[i]['npcoords'])
        vak_i = vak[i]
        for j , npcor in enumerate(vak_i['npcoords']):
            np_oplossing[npcor] = vak_i['kleur'] +  ' ' + str(vak_i['getallen'][j]) + ' '

    for i in range(len(np_oplossing)):
        print("".join(np_oplossing[i]))
    

In [44]:
def pri_alle_opl(rt = rt):
    len_opl = len(rt.oplossingen)
    for i in range(len_opl):
        print(printKleur['White'],'oplossing', i,  '\n')
        print_oplossing(model_nr = i)
        print()

pri_alle_opl()

 oplossing 0 

 1  2  5  2  5  3  1  4  1 
 3  4  3  4  1  4  5  3  2 
 1  5  2  5  3  2  1  4  1 
 2  3  1  4  1  5  3  5  3 
 1  4  2  5  2  4  2  4  2 
 3  5  3  1  3  1  5  1  3 
 4  2  4  2  5  2  4  2  4 
 5  1  5  1  4  3  5  3  1 
 2  3  4  3  5  2  1  4  2 



# De berekende oplossing van
<img src = '7julitub9x9.jpg'/>

```
 oplossing 0 

 1  2  5  2  5  3  1  4  1 
 3  4  3  4  1  4  5  3  2 
 1  5  2  5  3  2  1  4  1 
 2  3  1  4  1  5  3  5  3 
 1  4  2  5  2  4  2  4  2 
 3  5  3  1  3  1  5  1  3 
 4  2  4  2  5  2  4  2  4 
 5  1  5  1  4  3  5  3  1 
 2  3  4  3  5  2  1  4  2 
 ```
 